In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as mp

from scipy import stats
from scipy.stats import zscore
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression

In [2]:
csv = pd.read_csv("train.csv")
train = pd.DataFrame(csv)
train.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 87 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   CUST_ID                  700 non-null    object 
 1   INCOME                   700 non-null    int64  
 2   SAVINGS                  700 non-null    int64  
 3   DEBT                     700 non-null    int64  
 4   R_SAVINGS_INCOME         700 non-null    float64
 5   R_DEBT_INCOME            700 non-null    float64
 6   R_DEBT_SAVINGS           700 non-null    float64
 7   T_CLOTHING_12            700 non-null    int64  
 8   T_CLOTHING_6             700 non-null    int64  
 9   R_CLOTHING               700 non-null    float64
 10  R_CLOTHING_INCOME        700 non-null    float64
 11  R_CLOTHING_SAVINGS       700 non-null    float64
 12  R_CLOTHING_DEBT          700 non-null    float64
 13  T_EDUCATION_12           700 non-null    int64  
 14  T_EDUCATION_6            7

In [3]:
csv = pd.read_csv("test.csv")
test = pd.DataFrame(csv)
test

,CUST_ID,INCOME,SAVINGS,DEBT,R_SAVINGS_INCOME,R_DEBT_INCOME,R_DEBT_SAVINGS,T_CLOTHING_12,T_CLOTHING_6,R_CLOTHING,...,R_EXPENDITURE_INCOME,R_EXPENDITURE_SAVINGS,R_EXPENDITURE_DEBT,CAT_GAMBLING,CAT_DEBT,CAT_CREDIT_CARD,CAT_MORTGAGE,CAT_SAVINGS_ACCOUNT,CAT_DEPENDENTS,DEFAULT
0,2ece1c0c86d06134,81290,966445,162580,11.8889,2.0000,0.1682,1255,0,0.1268,...,1.1111,0.0935,0.5556,No,1,0,0,1,0,0
1,94f9e0d9a7335975,25397,12698,690798,0.5000,27.2000,54.4021,1065,726,0.6817,...,1.0000,2.0001,0.0368,No,1,0,0,1,0,1
2,9f577ce0fd5bc804,158349,906910,187140,5.7273,1.1818,0.2063,6546,3268,0.4992,...,0.9091,0.1587,0.7692,High,1,0,0,1,1,0
3,50e9b79060928e1a,181639,544916,1634749,3.0000,9.0000,3.0000,10144,4351,0.4289,...,0.7143,0.2381,0.0794,No,1,1,0,1,0,0
4,c5be52b42ad6658f,277118,1408685,3972030,5.0833,14.3334,2.8197,8739,5561,0.6363,...,0.8333,0.1639,0.0581,High,1,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,987b2eb86827ba65,307348,351254,1866039,1.1429,6.0714,5.3125,15277,4034,0.2641,...,0.7143,0.6250,0.1176,No,1,1,1,1,0,1
296,33429bfd04967f54,244600,838627,1118170,3.4286,4.5714,1.3333,14348,8273,0.5766,...,0.7143,0.2083,0.1562,No,1,0,1,1,0,1
297,ac001fd6510a0063,17371,129317,25091,7.4444,1.4444,0.1940,527,330,0.6262,...,1.1111,0.1493,0.7692,Low,1,0,0,1,0,0
298,214fb53907843cce,97081,153712,347874,1.5833,3.5833,2.2632,7903,4098,0.5185,...,0.8333,0.5263,0.2326,No,1,0,0,1,0,0


In [4]:
train.dtypes

CUST_ID                 object
INCOME                   int64
SAVINGS                  int64
DEBT                     int64
R_SAVINGS_INCOME       float64
                        ...   
CAT_MORTGAGE             int64
CAT_SAVINGS_ACCOUNT      int64
CAT_DEPENDENTS           int64
CREDIT_SCORE             int64
DEFAULT                  int64
Length: 87, dtype: object

In [5]:
# Define mapping for categories to integers
mapping = {'No': 0, 'Low': 1, 'High': 2}

# Convert categorical column to integers
train['CAT_GAMBLING_INT'] = train['CAT_GAMBLING'].map(mapping)
test['CAT_GAMBLING_INT'] = test['CAT_GAMBLING'].map(mapping)

# Drop the old column
train.drop(columns=['CAT_GAMBLING'], inplace=True)
test.drop(columns=['CAT_GAMBLING'], inplace=True)

# Convert columns to category type
train = train.astype({'CAT_GAMBLING_INT': 'category', 'DEFAULT': 'category',
                      'CAT_DEBT': 'category', 'CAT_CREDIT_CARD': 'category',
                      'CAT_MORTGAGE': 'category', 'CAT_SAVINGS_ACCOUNT': 'category',
                      'CAT_DEPENDENTS': 'category'})

test = test.astype({'CAT_GAMBLING_INT': 'category', 'DEFAULT': 'category',
                    'CAT_DEBT': 'category', 'CAT_CREDIT_CARD': 'category',
                    'CAT_MORTGAGE': 'category', 'CAT_SAVINGS_ACCOUNT': 'category',
                    'CAT_DEPENDENTS': 'category'})

# Check data types
print("Train Data Types:")
print(train.info(verbose=True))


print("\nTest Data Types:")
print(test.info(verbose=True))

Train Data Types:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 87 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   CUST_ID                  700 non-null    object  
 1   INCOME                   700 non-null    int64   
 2   SAVINGS                  700 non-null    int64   
 3   DEBT                     700 non-null    int64   
 4   R_SAVINGS_INCOME         700 non-null    float64 
 5   R_DEBT_INCOME            700 non-null    float64 
 6   R_DEBT_SAVINGS           700 non-null    float64 
 7   T_CLOTHING_12            700 non-null    int64   
 8   T_CLOTHING_6             700 non-null    int64   
 9   R_CLOTHING               700 non-null    float64 
 10  R_CLOTHING_INCOME        700 non-null    float64 
 11  R_CLOTHING_SAVINGS       700 non-null    float64 
 12  R_CLOTHING_DEBT          700 non-null    float64 
 13  T_EDUCATION_12           700 non-null    int64 

In [6]:
train.columns

Index(['CUST_ID', 'INCOME', 'SAVINGS', 'DEBT', 'R_SAVINGS_INCOME',
       'R_DEBT_INCOME', 'R_DEBT_SAVINGS', 'T_CLOTHING_12', 'T_CLOTHING_6',
       'R_CLOTHING', 'R_CLOTHING_INCOME', 'R_CLOTHING_SAVINGS',
       'R_CLOTHING_DEBT', 'T_EDUCATION_12', 'T_EDUCATION_6', 'R_EDUCATION',
       'R_EDUCATION_INCOME', 'R_EDUCATION_SAVINGS', 'R_EDUCATION_DEBT',
       'T_ENTERTAINMENT_12', 'T_ENTERTAINMENT_6', 'R_ENTERTAINMENT',
       'R_ENTERTAINMENT_INCOME', 'R_ENTERTAINMENT_SAVINGS',
       'R_ENTERTAINMENT_DEBT', 'T_FINES_12', 'T_FINES_6', 'R_FINES',
       'R_FINES_INCOME', 'R_FINES_SAVINGS', 'R_FINES_DEBT', 'T_GAMBLING_12',
       'T_GAMBLING_6', 'R_GAMBLING', 'R_GAMBLING_INCOME', 'R_GAMBLING_SAVINGS',
       'R_GAMBLING_DEBT', 'T_GROCERIES_12', 'T_GROCERIES_6', 'R_GROCERIES',
       'R_GROCERIES_INCOME', 'R_GROCERIES_SAVINGS', 'R_GROCERIES_DEBT',
       'T_HEALTH_12', 'T_HEALTH_6', 'R_HEALTH', 'R_HEALTH_INCOME',
       'R_HEALTH_SAVINGS', 'R_HEALTH_DEBT', 'T_HOUSING_12', 'T_HOUSING_6',
 

In [7]:
x_CV = train.drop(columns=['CUST_ID', 'CREDIT_SCORE'])
x_CV

,INCOME,SAVINGS,DEBT,R_SAVINGS_INCOME,R_DEBT_INCOME,R_DEBT_SAVINGS,T_CLOTHING_12,T_CLOTHING_6,R_CLOTHING,R_CLOTHING_INCOME,...,R_EXPENDITURE_INCOME,R_EXPENDITURE_SAVINGS,R_EXPENDITURE_DEBT,CAT_DEBT,CAT_CREDIT_CARD,CAT_MORTGAGE,CAT_SAVINGS_ACCOUNT,CAT_DEPENDENTS,DEFAULT,CAT_GAMBLING_INT
0,195779,307652,1468341,1.5714,7.5000,4.7727,11842,1541,0.1301,0.0605,...,0.7143,0.4545,0.0952,1,0,0,1,0,0,2
1,328892,1465066,5501471,4.4546,16.7273,3.7551,16701,10132,0.6067,0.0508,...,0.9091,0.2041,0.0543,1,1,1,1,1,0,2
2,86786,433928,899414,5.0000,10.3636,2.0727,4098,3167,0.7728,0.0472,...,0.9091,0.1818,0.0877,1,0,1,1,1,1,2
3,92747,176219,500834,1.9000,5.4000,2.8421,8748,3722,0.4255,0.0943,...,1.0000,0.5263,0.1852,1,1,0,1,0,0,0
4,55360,110720,437344,2.0000,7.9000,3.9500,3926,1105,0.2815,0.0709,...,1.0000,0.5000,0.1266,1,0,0,1,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,124492,572663,1904728,4.6000,15.3000,3.3261,5196,920,0.1771,0.0417,...,1.0000,0.2174,0.0654,1,0,0,1,1,0,0
696,337242,2422014,4046909,7.1818,12.0000,1.6709,9034,4187,0.4635,0.0268,...,0.9091,0.1266,0.0758,1,1,1,1,1,1,2
697,152243,478478,772090,3.1429,5.0714,1.6136,4225,1044,0.2471,0.0278,...,0.7143,0.2273,0.1408,1,0,0,1,0,0,2
698,188674,1021982,1871013,5.4167,9.9166,1.8308,5382,666,0.1237,0.0285,...,0.8333,0.1538,0.0840,1,1,1,1,1,0,2


In [8]:
y_CV = train['CREDIT_SCORE']
y_CV

0      575
1      418
2      521
3      606
4      575
      ... 
695    515
696    502
697    599
698    537
699    650
Name: CREDIT_SCORE, Length: 700, dtype: int64

In [9]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import BayesianRidge

# Define the number of folds
k_folds = 5

# Define the model
model = LinearRegression()
#model = SVR(kernel='linear')
#model = DecisionTreeRegressor()
#model = RandomForestRegressor()
#model = GradientBoostingRegressor()
#model = Ridge()
#model = ElasticNet()
#model = BayesianRidge()


# Perform k-fold cross-validation
scores = cross_val_score(model, x_CV, y_CV, cv=k_folds, scoring = 'neg_mean_squared_error')

# Print the scores for each fold
rmse = np.mean(np.sqrt(np.abs(scores)))
print("Scores for each fold:", rmse)

Scores for each fold: 29.18064661791932


In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

# Define preprocessing steps for numerical and categorical columns
numerical_features = ['INCOME', 'SAVINGS', 'DEBT', 'R_SAVINGS_INCOME',
       'R_DEBT_INCOME', 'R_DEBT_SAVINGS', 'T_CLOTHING_12', 'T_CLOTHING_6',
       'R_CLOTHING', 'R_CLOTHING_INCOME', 'R_CLOTHING_SAVINGS',
       'R_CLOTHING_DEBT', 'T_EDUCATION_12', 'T_EDUCATION_6', 'R_EDUCATION',
       'R_EDUCATION_INCOME', 'R_EDUCATION_SAVINGS', 'R_EDUCATION_DEBT',
       'T_ENTERTAINMENT_12', 'T_ENTERTAINMENT_6', 'R_ENTERTAINMENT',
       'R_ENTERTAINMENT_INCOME', 'R_ENTERTAINMENT_SAVINGS',
       'R_ENTERTAINMENT_DEBT', 'T_FINES_12', 'T_FINES_6', 'R_FINES',
       'R_FINES_INCOME', 'R_FINES_SAVINGS', 'R_FINES_DEBT', 'T_GAMBLING_12',
       'T_GAMBLING_6', 'R_GAMBLING', 'R_GAMBLING_INCOME', 'R_GAMBLING_SAVINGS',
       'R_GAMBLING_DEBT', 'T_GROCERIES_12', 'T_GROCERIES_6', 'R_GROCERIES',
       'R_GROCERIES_INCOME', 'R_GROCERIES_SAVINGS', 'R_GROCERIES_DEBT',
       'T_HEALTH_12', 'T_HEALTH_6', 'R_HEALTH', 'R_HEALTH_INCOME',
       'R_HEALTH_SAVINGS', 'R_HEALTH_DEBT', 'T_HOUSING_12', 'T_HOUSING_6',
       'R_HOUSING', 'R_HOUSING_INCOME', 'R_HOUSING_SAVINGS', 'R_HOUSING_DEBT',
       'T_TAX_12', 'T_TAX_6', 'R_TAX', 'R_TAX_INCOME', 'R_TAX_SAVINGS',
       'R_TAX_DEBT', 'T_TRAVEL_12', 'T_TRAVEL_6', 'R_TRAVEL',
       'R_TRAVEL_INCOME', 'R_TRAVEL_SAVINGS', 'R_TRAVEL_DEBT',
       'T_UTILITIES_12', 'T_UTILITIES_6', 'R_UTILITIES', 'R_UTILITIES_INCOME',
       'R_UTILITIES_SAVINGS', 'R_UTILITIES_DEBT', 'T_EXPENDITURE_12',
       'T_EXPENDITURE_6', 'R_EXPENDITURE', 'R_EXPENDITURE_INCOME',
       'R_EXPENDITURE_SAVINGS', 'R_EXPENDITURE_DEBT']


numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())  # Adding polynomial features
])


# Combine preprocessing steps for all types of features
preprocessor = ColumnTransformer(transformers=[('num', numerical_transformer, numerical_features)])

# Create a pipeline with preprocessing and model fitting
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', Ridge())])

# # Fit the pipeline to the training data
# pipeline.fit(X_train, y_train)

# # Evaluate model performance
# train_rmse = mean_squared_error(y_train, pipeline.predict(X_train), squared=False)
# test_rmse = mean_squared_error(y_test, pipeline.predict(X_test), squared=False)

# print("Train RMSE:", train_rmse)
# print("Test RMSE:", test_rmse)

# Define the number of folds
k_folds = 5

# Perform k-fold cross-validation
scores = cross_val_score(pipeline, x_CV, y_CV, cv=k_folds, scoring = 'neg_mean_squared_error')

# Print the scores for each fold
rmse = np.mean(np.sqrt(np.abs(scores)))
print("Scores for each fold:", rmse)

Scores for each fold: 29.505158872294277


In [11]:
# Set column 'CUST_ID' as index
train.set_index('CUST_ID', inplace=True)
test.set_index('CUST_ID', inplace=True)
train

,INCOME,SAVINGS,DEBT,R_SAVINGS_INCOME,R_DEBT_INCOME,R_DEBT_SAVINGS,T_CLOTHING_12,T_CLOTHING_6,R_CLOTHING,R_CLOTHING_INCOME,...,R_EXPENDITURE_SAVINGS,R_EXPENDITURE_DEBT,CAT_DEBT,CAT_CREDIT_CARD,CAT_MORTGAGE,CAT_SAVINGS_ACCOUNT,CAT_DEPENDENTS,CREDIT_SCORE,DEFAULT,CAT_GAMBLING_INT
CUST_ID,,,,,,,,,,,,,,,,,,,,,
17cef92dcaa72204,195779,307652,1468341,1.5714,7.5000,4.7727,11842,1541,0.1301,0.0605,...,0.4545,0.0952,1,0,0,1,0,575,0,2
d5686fcfa23294cd,328892,1465066,5501471,4.4546,16.7273,3.7551,16701,10132,0.6067,0.0508,...,0.2041,0.0543,1,1,1,1,1,418,0,2
927fa6bd39cc2dc1,86786,433928,899414,5.0000,10.3636,2.0727,4098,3167,0.7728,0.0472,...,0.1818,0.0877,1,0,1,1,1,521,1,2
ec34b6662956e833,92747,176219,500834,1.9000,5.4000,2.8421,8748,3722,0.4255,0.0943,...,0.5263,0.1852,1,1,0,1,0,606,0,0
f4c9d3c223fbc054,55360,110720,437344,2.0000,7.9000,3.9500,3926,1105,0.2815,0.0709,...,0.5000,0.1266,1,0,0,1,0,575,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
a9f968f56226bc48,124492,572663,1904728,4.6000,15.3000,3.3261,5196,920,0.1771,0.0417,...,0.2174,0.0654,1,0,0,1,1,515,0,0
80cd6b5ac5e17831,337242,2422014,4046909,7.1818,12.0000,1.6709,9034,4187,0.4635,0.0268,...,0.1266,0.0758,1,1,1,1,1,502,1,2
b56fc25c694eb83d,152243,478478,772090,3.1429,5.0714,1.6136,4225,1044,0.2471,0.0278,...,0.2273,0.1408,1,0,0,1,0,599,0,2


In [12]:
X_train = train.drop(columns=['CREDIT_SCORE', 'DEFAULT'])
X_train

,INCOME,SAVINGS,DEBT,R_SAVINGS_INCOME,R_DEBT_INCOME,R_DEBT_SAVINGS,T_CLOTHING_12,T_CLOTHING_6,R_CLOTHING,R_CLOTHING_INCOME,...,R_EXPENDITURE,R_EXPENDITURE_INCOME,R_EXPENDITURE_SAVINGS,R_EXPENDITURE_DEBT,CAT_DEBT,CAT_CREDIT_CARD,CAT_MORTGAGE,CAT_SAVINGS_ACCOUNT,CAT_DEPENDENTS,CAT_GAMBLING_INT
CUST_ID,,,,,,,,,,,,,,,,,,,,,
17cef92dcaa72204,195779,307652,1468341,1.5714,7.5000,4.7727,11842,1541,0.1301,0.0605,...,0.3687,0.7143,0.4545,0.0952,1,0,0,1,0,2
d5686fcfa23294cd,328892,1465066,5501471,4.4546,16.7273,3.7551,16701,10132,0.6067,0.0508,...,0.5193,0.9091,0.2041,0.0543,1,1,1,1,1,2
927fa6bd39cc2dc1,86786,433928,899414,5.0000,10.3636,2.0727,4098,3167,0.7728,0.0472,...,0.5144,0.9091,0.1818,0.0877,1,0,1,1,1,2
ec34b6662956e833,92747,176219,500834,1.9000,5.4000,2.8421,8748,3722,0.4255,0.0943,...,0.5353,1.0000,0.5263,0.1852,1,1,0,1,0,0
f4c9d3c223fbc054,55360,110720,437344,2.0000,7.9000,3.9500,3926,1105,0.2815,0.0709,...,0.5328,1.0000,0.5000,0.1266,1,0,0,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
a9f968f56226bc48,124492,572663,1904728,4.6000,15.3000,3.3261,5196,920,0.1771,0.0417,...,0.4715,1.0000,0.2174,0.0654,1,0,0,1,1,0
80cd6b5ac5e17831,337242,2422014,4046909,7.1818,12.0000,1.6709,9034,4187,0.4635,0.0268,...,0.5817,0.9091,0.1266,0.0758,1,1,1,1,1,2
b56fc25c694eb83d,152243,478478,772090,3.1429,5.0714,1.6136,4225,1044,0.2471,0.0278,...,0.4125,0.7143,0.2273,0.1408,1,0,0,1,0,2


In [13]:
y_train = train['CREDIT_SCORE']
y_train

CUST_ID
17cef92dcaa72204    575
d5686fcfa23294cd    418
927fa6bd39cc2dc1    521
ec34b6662956e833    606
f4c9d3c223fbc054    575
                   ... 
a9f968f56226bc48    515
80cd6b5ac5e17831    502
b56fc25c694eb83d    599
ac254de609c6fe5f    537
3d40082c3c8c575f    650
Name: CREDIT_SCORE, Length: 700, dtype: int64

In [14]:
#X_test = pd.DataFrame(test)
X_test = test.drop(columns=['DEFAULT'])
X_test

,INCOME,SAVINGS,DEBT,R_SAVINGS_INCOME,R_DEBT_INCOME,R_DEBT_SAVINGS,T_CLOTHING_12,T_CLOTHING_6,R_CLOTHING,R_CLOTHING_INCOME,...,R_EXPENDITURE,R_EXPENDITURE_INCOME,R_EXPENDITURE_SAVINGS,R_EXPENDITURE_DEBT,CAT_DEBT,CAT_CREDIT_CARD,CAT_MORTGAGE,CAT_SAVINGS_ACCOUNT,CAT_DEPENDENTS,CAT_GAMBLING_INT
CUST_ID,,,,,,,,,,,,,,,,,,,,,
2ece1c0c86d06134,81290,966445,162580,11.8889,2.0000,0.1682,1255,0,0.1268,0.0154,...,0.5455,1.1111,0.0935,0.5556,1,0,0,1,0,0
94f9e0d9a7335975,25397,12698,690798,0.5000,27.2000,54.4021,1065,726,0.6817,0.0419,...,0.5248,1.0000,2.0001,0.0368,1,0,0,1,0,0
9f577ce0fd5bc804,158349,906910,187140,5.7273,1.1818,0.2063,6546,3268,0.4992,0.0413,...,0.5260,0.9091,0.1587,0.7692,1,0,0,1,1,2
50e9b79060928e1a,181639,544916,1634749,3.0000,9.0000,3.0000,10144,4351,0.4289,0.0558,...,0.4357,0.7143,0.2381,0.0794,1,1,0,1,0,0
c5be52b42ad6658f,277118,1408685,3972030,5.0833,14.3334,2.8197,8739,5561,0.6363,0.0315,...,0.5232,0.8333,0.1639,0.0581,1,1,0,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987b2eb86827ba65,307348,351254,1866039,1.1429,6.0714,5.3125,15277,4034,0.2641,0.0497,...,0.3593,0.7143,0.6250,0.1176,1,1,1,1,0,0
33429bfd04967f54,244600,838627,1118170,3.4286,4.5714,1.3333,14348,8273,0.5766,0.0587,...,0.6207,0.7143,0.2083,0.1562,1,0,1,1,0,0
ac001fd6510a0063,17371,129317,25091,7.4444,1.4444,0.1940,527,330,0.6262,0.0303,...,0.4995,1.1111,0.1493,0.7692,1,0,0,1,0,1


In [15]:
X_test.dtypes

INCOME                    int64
SAVINGS                   int64
DEBT                      int64
R_SAVINGS_INCOME        float64
R_DEBT_INCOME           float64
                         ...   
CAT_CREDIT_CARD        category
CAT_MORTGAGE           category
CAT_SAVINGS_ACCOUNT    category
CAT_DEPENDENTS         category
CAT_GAMBLING_INT       category
Length: 84, dtype: object

In [16]:
# # Importing necessary libraries
# from sklearn.datasets import make_regression
# from sklearn.model_selection import train_test_split
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.metrics import mean_squared_error

# # Creating and training the Decision Tree Regression model
# tree_reg = DecisionTreeRegressor(max_depth=5)  # You can adjust max_depth for controlling the complexity
# tree_reg.fit(X_train, y_train)

# # Making predictions on the test set
# y_pred = tree_reg.predict(X_test)
# y_pred

In [17]:
# Define preprocessing steps for numerical and categorical columns
numerical_features = ['INCOME', 'SAVINGS', 'DEBT', 'R_SAVINGS_INCOME',
       'R_DEBT_INCOME', 'R_DEBT_SAVINGS', 'T_CLOTHING_12', 'T_CLOTHING_6',
       'R_CLOTHING', 'R_CLOTHING_INCOME', 'R_CLOTHING_SAVINGS',
       'R_CLOTHING_DEBT', 'T_EDUCATION_12', 'T_EDUCATION_6', 'R_EDUCATION',
       'R_EDUCATION_INCOME', 'R_EDUCATION_SAVINGS', 'R_EDUCATION_DEBT',
       'T_ENTERTAINMENT_12', 'T_ENTERTAINMENT_6', 'R_ENTERTAINMENT',
       'R_ENTERTAINMENT_INCOME', 'R_ENTERTAINMENT_SAVINGS',
       'R_ENTERTAINMENT_DEBT', 'T_FINES_12', 'T_FINES_6', 'R_FINES',
       'R_FINES_INCOME', 'R_FINES_SAVINGS', 'R_FINES_DEBT', 'T_GAMBLING_12',
       'T_GAMBLING_6', 'R_GAMBLING', 'R_GAMBLING_INCOME', 'R_GAMBLING_SAVINGS',
       'R_GAMBLING_DEBT', 'T_GROCERIES_12', 'T_GROCERIES_6', 'R_GROCERIES',
       'R_GROCERIES_INCOME', 'R_GROCERIES_SAVINGS', 'R_GROCERIES_DEBT',
       'T_HEALTH_12', 'T_HEALTH_6', 'R_HEALTH', 'R_HEALTH_INCOME',
       'R_HEALTH_SAVINGS', 'R_HEALTH_DEBT', 'T_HOUSING_12', 'T_HOUSING_6',
       'R_HOUSING', 'R_HOUSING_INCOME', 'R_HOUSING_SAVINGS', 'R_HOUSING_DEBT',
       'T_TAX_12', 'T_TAX_6', 'R_TAX', 'R_TAX_INCOME', 'R_TAX_SAVINGS',
       'R_TAX_DEBT', 'T_TRAVEL_12', 'T_TRAVEL_6', 'R_TRAVEL',
       'R_TRAVEL_INCOME', 'R_TRAVEL_SAVINGS', 'R_TRAVEL_DEBT',
       'T_UTILITIES_12', 'T_UTILITIES_6', 'R_UTILITIES', 'R_UTILITIES_INCOME',
       'R_UTILITIES_SAVINGS', 'R_UTILITIES_DEBT', 'T_EXPENDITURE_12',
       'T_EXPENDITURE_6', 'R_EXPENDITURE', 'R_EXPENDITURE_INCOME',
       'R_EXPENDITURE_SAVINGS', 'R_EXPENDITURE_DEBT']


numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())  # Adding polynomial features
])


# Combine preprocessing steps for all types of features
preprocessor = ColumnTransformer(transformers=[('num', numerical_transformer, numerical_features)])

# Create a pipeline with preprocessing and model fitting
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', Ridge())])

# # Fit the pipeline to the training data
pipeline.fit(X_train, y_train)
yhat = pipeline.predict(X_test)
yhat

array([630.27788726, 409.59497795, 654.4166452 , 584.77680008,
       520.81430192, 576.28845215, 616.48689533, 531.01640667,
       596.89618267, 603.75879012, 617.96373629, 630.92745492,
       591.02888807, 579.28751928, 477.60957103, 608.40123497,
       612.44345791, 663.49168535, 598.19803181, 589.21872132,
       577.36244628, 564.17559525, 636.08687465, 418.28315877,
       607.31041166, 603.66970083, 604.69875513, 641.88232161,
       637.93558074, 556.39365185, 427.58893473, 580.35211036,
       614.01495418, 544.75932491, 567.42215916, 621.65111254,
       593.50264314, 662.86884531, 532.11795501, 611.1372027 ,
       634.2617971 , 650.73913188, 594.07559143, 634.16786808,
       653.97391703, 617.13301547, 574.39248915, 610.91391209,
       539.51274367, 706.13790047, 593.02284873, 544.89236149,
       568.68809281, 600.52805253, 635.99757571, 657.35376669,
       555.73106829, 540.05724303, 539.56921238, 496.59659258,
       551.6649481 , 596.56756764, 556.29470561, 623.08

In [18]:
model = Ridge()
model.fit(X_train, y_train)
yhat = model.predict(X_test)
yhat

array([632.08851985, 402.31633422, 638.4023656 , 582.83718713,
       515.28100705, 569.43434779, 608.90392552, 545.71148363,
       598.8112881 , 596.48048345, 616.74415179, 631.87770716,
       586.90519699, 579.05926982, 487.58879079, 619.26753996,
       610.80655577, 670.19321495, 583.36206125, 584.07371108,
       583.96041746, 543.36756671, 636.4470878 , 409.91289894,
       607.70375127, 608.62094474, 615.18434328, 644.2813368 ,
       637.2741306 , 544.77995448, 437.17273885, 576.59729191,
       612.05505324, 539.83147269, 573.86049033, 614.99507413,
       581.03719258, 653.32038988, 545.42943898, 611.97797333,
       621.67139712, 649.35671308, 586.12517872, 632.3206235 ,
       639.48291375, 608.57474308, 568.48405867, 579.60321124,
       548.36154911, 679.75697914, 599.5850092 , 545.05664597,
       567.05070746, 609.08501096, 632.34915112, 648.3685191 ,
       558.26702863, 557.24991879, 519.76435655, 497.58118872,
       542.52323002, 580.98617751, 556.8678017 , 616.78

In [19]:
predictions_df = pd.DataFrame(yhat, columns=['CREDIT_SCORE'], index=X_test.index)
predictions_df

,CREDIT_SCORE
CUST_ID,
2ece1c0c86d06134,632.088520
94f9e0d9a7335975,402.316334
9f577ce0fd5bc804,638.402366
50e9b79060928e1a,582.837187
c5be52b42ad6658f,515.281007
...,...
987b2eb86827ba65,625.634991
33429bfd04967f54,603.087549
ac001fd6510a0063,610.629956


In [20]:
predictions_df.to_csv('credScore.csv')